# Analysis

Here, we analyze the results from the PIA experiment. Specifically, we look at the F1-Score, Precision, Recall, and Confusion Matrix.

In [1]:
import json
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [2]:
with open("data/formatted_results/organized_results.json", "r") as f:
    organized_data = json.load(f)

In [4]:
key = list(organized_data.keys())[0]

gt_email = [False] * len(organized_data[key]['email'])
gt_table = [False] * len(organized_data[key]['table'])
gt_benign = [True] * len(organized_data[key]['benign'])

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

for key in organized_data.keys():
    dataset = organized_data[key]

    print("------EMAIL RESULTS-------")
    email_results = dataset['email'] + dataset['benign']
    gt_results = gt_email + gt_benign
    print(key)
    print("F1 Score: {}".format(f1_score(gt_results, email_results, pos_label=False)))
    print("Precision: {}".format(precision_score(gt_results, email_results, pos_label=False)))
    print("Recall: {}".format(recall_score(gt_results, email_results, pos_label=False)))
    print("Confusion Matrix: \n\n {} \n".format(confusion_matrix(gt_results, email_results, labels=[False, True])))

    print("------TABLE RESULTS-------")
    table_results = dataset['table'] + dataset['benign']
    gt_results = gt_table + gt_benign
    print(key)
    print("F1 Score: {}".format(f1_score(gt_results, table_results, pos_label=False)))
    print("Precision: {}".format(precision_score(gt_results, table_results,  pos_label=False)))
    print("Recall: {}".format(recall_score(gt_results, table_results, pos_label=False)))
    print("Confusion Matrix: \n\n {} \n".format(confusion_matrix(gt_results, table_results, labels=[False, True])))

------EMAIL RESULTS-------
ProtectAI_deberta_v3_base_prompt_injection_v2
F1 Score: 0.6321148065548337
Precision: 0.6952858361774744
Recall: 0.5794666666666667
Confusion Matrix: 

 [[6519 4731]
 [2857 8391]] 

------TABLE RESULTS-------
ProtectAI_deberta_v3_base_prompt_injection_v2
F1 Score: 0.0741275206015266
Precision: 0.2546308374641273
Recall: 0.04337777777777778
Confusion Matrix: 

 [[  976 21524]
 [ 2857  8391]] 

------EMAIL RESULTS-------
deepset_deberta_v3_base_injection
F1 Score: 0.7805995004163198
Precision: 0.6401502219187436
Recall: 1.0
Confusion Matrix: 

 [[11250     0]
 [ 6324  4924]] 

------TABLE RESULTS-------
deepset_deberta_v3_base_injection
F1 Score: 0.8767827916764087
Precision: 0.7805995004163198
Recall: 1.0
Confusion Matrix: 

 [[22500     0]
 [ 6324  4924]] 

------EMAIL RESULTS-------
ProtectAI_deberta_v3_small_prompt_injection_v2
F1 Score: 0.2409760414691194
Precision: 0.41715911578025827
Recall: 0.16942222222222222
Confusion Matrix: 

 [[1906 9344]
 [2663 85